## Download a Drilling Campaign object and save it in CSV format

This example shows how to download a drilling-campaign object from an Evo workspace and how to construct CSV files from the data.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
from pathlib import Path

import pandas as pd

from evo.notebooks import FeedbackWidget, ServiceManagerWidget
from evo.objects import ObjectAPIClient

cache_location = "data"

# Evo app credentials
client_id = "<your-client-id>"
redirect_url = "<your-redirect-url>"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### List all objects in the workspace.

In [ ]:
from prettytable import PrettyTable

all_objects = await object_client.list_all_objects()

table = PrettyTable(["Name", "Object ID"])
for index, obj in enumerate(all_objects):
    if "drilling-campaign" in obj.schema_id.sub_classification:
        table.add_row([obj.name.ljust(40), str(obj.id).ljust(40)])

if len(table.rows) == 0:
    print("No drilling campaigns found. Publish a drilling campaign using the 'publish-drilling-campaign' notebook.")
else:
    print("Drilling campaigns found:")
    print(table)

Enter the `Object ID` value for the chosen drilling-campaign object in the cell below.

In [ ]:
object_id = "<your-object-id>"

Download the Parquet files and assemble the CSV files.

In [ ]:
downloaded_object = await object_client.download_object_by_id(object_id=object_id)

metadata = downloaded_object.metadata
downloaded_dict = downloaded_object.as_dict()

df = pd.DataFrame()


def download_table(table_info, fb=None):
    return data_client.download_table(
        object_id=metadata.id, version_id=metadata.version_id, table_info=table_info, fb=fb
    )


# Use the data client to download the coordinate data.
hole_indices = (
    await download_table(
        downloaded_dict["hole_id"]["values"],
        fb=FeedbackWidget(f"Downloading hole indices data as '{downloaded_dict['hole_id']['values']['data']}'"),
    )
).to_pandas()
index_to_name_map = (
    await download_table(
        downloaded_dict["hole_id"]["table"],
        fb=FeedbackWidget(
            f"Downloading hole index to name map data as '{downloaded_dict['hole_id']['table']['data']}'"
        ),
    )
).to_pandas()

names = pd.DataFrame({"key": hole_indices["data"]}).merge(index_to_name_map, on="key", how="left")["value"]
names = names.rename("hole_id")

collar_locations = (
    await download_table(
        downloaded_dict["planned"]["collar"]["coordinates"],
        fb=FeedbackWidget(
            f"Downloading collar locations data as '{downloaded_dict['planned']['collar']['coordinates']['data']}'"
        ),
    )
).to_pandas()
hole_lengths = (
    await download_table(
        downloaded_dict["planned"]["collar"]["distances"],
        fb=FeedbackWidget(
            f"Downloading collar hole distances table as '{downloaded_dict['planned']['collar']['distances']['data']}'"
        ),
    )
).to_pandas()
chunk_data = (
    await download_table(
        downloaded_dict["planned"]["collar"]["holes"],
        fb=FeedbackWidget(
            f"Downloading collar chunks data as '{downloaded_dict['planned']['collar']['holes']['data']}'"
        ),
    )
).to_pandas()

attributes = []
# Use the data client to download the attribute data and merge it with the coordinates data.
if "attributes" in downloaded_dict["planned"]["collar"]:
    for attribute in downloaded_dict["planned"]["collar"]["attributes"]:
        attribute_name = attribute["name"]
        attribute_type = attribute["attribute_type"]

        # Download the attribute data. Every attribute has a 'values' data file.
        values_data = (
            await data_client.download_table(
                object_id=metadata.id,
                version_id=metadata.version_id,
                table_info=attribute["values"],
                fb=FeedbackWidget(
                    f"Downloading attribute '{attribute_name}' values data as '{attribute['values']['data']}'"
                ),
            )
        ).to_pandas()
        values_data = values_data.rename(columns={"data": attribute_name})
        attributes.append(values_data)

df = pd.concat([names, collar_locations, hole_lengths, *attributes], axis=1)
output_filename = Path(f"{cache_location}/collar.csv").resolve()
df.to_csv(output_filename, index=False)
print("Collar data saved to:", output_filename)

## Path
path_data = (
    await download_table(
        downloaded_dict["planned"]["path"],
        fb=FeedbackWidget(f"Downloading collar path data as '{downloaded_dict['planned']['path']['data']}'"),
    )
).to_pandas()

processed_path_data = pd.concat(
    [
        path_data.iloc[start : start + length].reset_index(drop=True)
        for start, length in zip(chunk_data["offset"], chunk_data["count"])
    ],
    axis=0,
).reset_index(drop=True)

hole_name = []

for hole_index, count in zip(chunk_data["hole_index"], chunk_data["count"]):
    hole_name.extend([index_to_name_map[index_to_name_map["key"] == hole_index]["value"].values[0]] * count)
processed_path_data["hole_name"] = hole_name
processed_path_data = processed_path_data[
    ["hole_name"] + [col for col in processed_path_data.columns if col != "hole_name"]
]

output_filename = Path(f"{cache_location}/path.csv").resolve()
processed_path_data.to_csv(output_filename, index=False)
print("Path data saved to:", output_filename)

## Collections
attribute_tables = {}
# Use the data client to download the attribute data and merge it with the coordinates data.
if "collections" in downloaded_dict["planned"]:
    for attribute_table in downloaded_dict["planned"]["collections"]:
        collection_name = attribute_table["name"]
        collection_type = attribute_table["collection_type"]

        if collection_type == "interval":
            distance_container = attribute_table["from_to"]["intervals"]
            attribute_container = attribute_table["from_to"]["attributes"]
            distance_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=distance_container,
                    fb=FeedbackWidget(f"Downloading distance data as '{distance_container['data']}'"),
                )
            ).to_pandas()
        elif collection_type == "distance":
            distance_container = attribute_table["distance"]["values"]
            attribute_container = attribute_table["distance"]["attributes"]
            distance_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=distance_container,
                    fb=FeedbackWidget(f"Downloading distance data as '{distance_container['data']}'"),
                )
            ).to_pandas()
        else:
            continue

        columns = [distance_data]
        for column in attribute_container:
            attribute_name = column["name"]
            attribute_type = column["attribute_type"]

            # If the attribute is a category, download the 'table' data as well.
            if attribute_type == "category":
                table_data = await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=attribute["table"],
                    fb=FeedbackWidget(
                        f"Downloading attribute '{attribute_name}' table data as '{attribute['table']['data']}'"
                    ),
                )

            # Download the attribute data. Every attribute has a 'values' data file.
            values_data = (
                await data_client.download_table(
                    object_id=metadata.id,
                    version_id=metadata.version_id,
                    table_info=column["values"],
                    fb=FeedbackWidget(
                        f"Downloading attribute '{attribute_name}' values data as '{column['values']['data']}'"
                    ),
                )
            ).to_pandas()

            # Assemble the dataframe.
            if attribute_type == "category":
                # Merge the values data with the table data.
                merged_data = pd.merge(values_data, table_data.to_pandas(), left_on="data", right_on="key", how="left")
                # Drop the 'data' and 'key' columns from the merged data.
                merged_data.drop(columns=["data", "key"], inplace=True)
                # Rename the 'value' column to the attribute name.
                merged_data.rename(columns={"value": attribute_name}, inplace=True)
                # Concatenate the merged data with the coordinates data.
                columns.append(merged_data)

            elif attribute_type == "scalar":
                data = values_data
                # Rename the 'data' column to the attribute name.
                data.rename(columns={"data": attribute_name}, inplace=True)
                # Concatenate the data with the coordinates data.
                columns.append(data)

        merged_df = pd.concat(columns, axis=1)
        processed_merged_df = pd.concat(
            [
                merged_df.iloc[start : start + length].reset_index(drop=True)
                for start, length in zip(chunk_data["offset"], chunk_data["count"])
            ],
            axis=0,
        ).reset_index(drop=True)

        hole_name = []
        for hole_index, count in zip(chunk_data["hole_index"], chunk_data["count"]):
            hole_name.extend([index_to_name_map[index_to_name_map["key"] == hole_index]["value"].values[0]] * count)
        processed_merged_df["hole_name"] = hole_name
        processed_merged_df = processed_merged_df[
            ["hole_name"] + [col for col in processed_merged_df.columns if col != "hole_name"]
        ]
        attribute_tables[collection_name] = processed_merged_df

if attribute_tables:
    for attribute in attribute_tables:
        output_filename = Path(f"{cache_location}/{attribute}.csv").resolve()
        attribute_tables[attribute].to_csv(output_filename, index=False)
        print(f"Collection '{attribute}' data saved to:", output_filename)

In [ ]:
from pathlib import Path

# Get the list of files in the cache_location directory (including subdirectories)
downloaded_files = Path(cache_location).glob("**/*")

# Iterate through each file and rename to add '.parquet' extension
for file_path in downloaded_files:
    if (
        file_path.is_file()
        and not file_path.name.endswith(".parquet")
        and not file_path.name.startswith(".")
        and not file_path.suffix
    ):  # Only rename files with no extension
        new_path = file_path.with_suffix(file_path.suffix + ".parquet")
        file_path.rename(new_path)
        print(f"Renamed: {file_path.name} -> {new_path.name}")